In [1]:
import argparse
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm

import torchvision.transforms as T
import torch
from torch import nn
from torch.utils.data import DataLoader
#DataLoader를 통해 네트워크에 올림
from utils.dataset import *
from utils.iou import IoU
from utils.Loss import *

from models.Unet import UNet
from models.ResUnet import ResUNet


In [2]:
torch.cuda.is_available()
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 4060 Laptop GPU'

In [3]:
class pytorchDataset():
    def __init__(self):
        # 요 데이터셋이 무엇을 사용할건지 준비
        pass

    def __len__(self):
        # 데이터셋의 길이를 측정할 수 있는 변수를 생성
        pass
    
    def __getitem__(self, idx):
        # 실제로 어떻게 구동하는지
        # idx -> 데이터셋의 index 
        # 데이터셋의 길이를 따로 설정
        # collate_fn
        pass

In [4]:
#데이터 불러오는 클래스 지정
class SegCTDataset(Dataset):
    """Covid XRay dataset."""

    def __init__(self, image, mask, txt, transforms):
        # 요 데이터셋이 무엇을 사용할건지 준비
        # 이니셜라이징
        self.IMAGE_LIB = image      # Input 이미지를 뽑을 폴더
        self.MASK_LIB = mask        # Ouput 이미지를 뽑을 폴더
        self.images = np.loadtxt(txt, dtype=str)    # 각 뽑아낸 이미지가 train/text에 속하는지
        self.transform = transforms # 뽑아낸 이미지를 어떻게 변환시킬 것인지

    def __len__(self):
        return len(self.images)

##까지 저절로 구동됨 
    
    def __getitem__(self, idx):
        #__getitem__을 통해 특정 인덱스의 아이템을 출력
        # 에러방지
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_name = self.images[idx]
            # 0: ID_0041_Z_0336.tif
            # 1: ID_0107_Z_0062.tif
            # 2: ID_0106_Z_0130.tif

        # print(self.IMAGE_LIB + image_name)
        # print(cv2.imread(self.IMAGE_LIB + image_name, cv2.IMREAD_UNCHANGED))
        
        img = cv2.imread(self.IMAGE_LIB + image_name, cv2.IMREAD_UNCHANGED).astype("int16").astype('float32')
            # 이미지 디렉토리 + 파일네임 불러오겠다
            # opencv로 이미지를 불러오면 numpy
        
        img = (img - np.min(img)) / (np.max(img) - np.min(img))
            # 직접 0-1 정규화를 수행
        
        img = Image.fromarray(np.uint8(img * 255)).convert('RGB')
            # numpy -> PIL image
            # transform을 위해서 변환하기
        
        label = Image.open(self.MASK_LIB + image_name)
            # PIL image
            # Segmentation task는 Input/Output에 모두 같은 '전처리' 수행

        if self.transform:
            img, label = self.transform((img, label))

        sample = {'img': img,'label': label}
        return sample

In [5]:
#데이터 변환관련
#이미지 전처리 
train_transform = transforms.Compose([
        RandomRescale(0.6,1.5),
        RandomCrop((320, 320)),
        RandomFlip(),
        RandomColor(),
        ToTensor(),
        # Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

test_transform = transforms.Compose([
        Resize((320, 400)),
        ToTensor(),
        # Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

In [6]:
# 1. 데이터 준비
image_dir   = './archive/2d_images/'
mask_dir    = './archive/2d_masks/'
train_txt   = './train.txt'

dataSet = SegCTDataset(image = image_dir, mask=mask_dir, txt=train_txt, transforms=train_transform)
train_size = int(0.8 * len(dataSet))
valid_size = len(dataSet) - train_size
trainSet, validSet = torch.utils.data.random_split(dataSet, [train_size, valid_size])

train_loader = DataLoader(trainSet, batch_size=2, shuffle=True)
val_loader = DataLoader(validSet, batch_size=2)

In [7]:
dataSet[0]


{'img': tensor([[[0.5098, 0.5098, 0.5059,  ..., 0.5137, 0.4902, 0.4941],
          [0.4392, 0.4627, 0.4941,  ..., 0.4863, 0.5137, 0.4824],
          [0.4588, 0.4275, 0.4667,  ..., 0.4314, 0.4863, 0.4784],
          ...,
          [0.6078, 0.6667, 0.6863,  ..., 0.4941, 0.4980, 0.4588],
          [0.6510, 0.6588, 0.6235,  ..., 0.5137, 0.4353, 0.4588],
          [0.6431, 0.6392, 0.5843,  ..., 0.5059, 0.4588, 0.4863]],
 
         [[0.5098, 0.5098, 0.5059,  ..., 0.5137, 0.4902, 0.4941],
          [0.4392, 0.4627, 0.4941,  ..., 0.4863, 0.5137, 0.4824],
          [0.4588, 0.4275, 0.4667,  ..., 0.4314, 0.4863, 0.4784],
          ...,
          [0.6078, 0.6667, 0.6863,  ..., 0.4941, 0.4980, 0.4588],
          [0.6510, 0.6588, 0.6235,  ..., 0.5137, 0.4353, 0.4588],
          [0.6431, 0.6392, 0.5843,  ..., 0.5059, 0.4588, 0.4863]],
 
         [[0.5098, 0.5098, 0.5059,  ..., 0.5137, 0.4902, 0.4941],
          [0.4392, 0.4627, 0.4941,  ..., 0.4863, 0.5137, 0.4824],
          [0.4588, 0.4275, 0.4667

In [8]:
#훈련 함수 지정
def train(model, train_loader, optimizer, LOSS_FUNC, EPOCH, PRINT_INTERVAL, epoch):
    losses = []
    for i, batch in enumerate(tqdm(train_loader)):
        
        img, label = batch['img'], batch['label']
        ## 
        output = model(img)
        loss = LOSS_FUNC(output, label)     # Dice loss

        optimizer.zero_grad()
        loss.backward()     # 모델의 어느 부분이 못했는지
        optimizer.step()    # 모델의 못한부분을 실제로 바꿔주기

        losses.append(loss.item())
        if (i + 1) % PRINT_INTERVAL == 0:
            tqdm.write('Epoch [%d/%d], Iter [%d/%d], Loss: %.4f'
                       % (epoch + 1, EPOCH, i + 1, len(train_loader), loss.item()))
    return np.mean(losses)

#validation 함수 지정
def eval(model,val_loader,LOSS_FUNC):
    losses = []
    for i, batch in enumerate(val_loader):
        img, label = batch['img'], batch['label']
        output = model(img)
        loss = LOSS_FUNC(output, label)
        losses.append(loss.item())
    return np.mean(losses)

In [9]:
# 2. 모델 준비
model = ResUNet()

# 3. 학습 준비
LOSS_FUNC = DiceCELoss() # Dice score
PRINT_INTERVAL = 5
EPOCH = 100
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

val_loss_epoch = []

    # 학습시작
for epoch in range(EPOCH):

    model.train()   # 해당 모델은 train 하는 중이다
    train_loss = train(model, train_loader, optimizer, LOSS_FUNC, EPOCH, PRINT_INTERVAL, epoch)
    # train_loss > 전체 데이터셋에서 loss의 평균값
    
    #validation 시작
    model.eval()
    val_loss = eval(model, val_loader, LOSS_FUNC)
    val_loss_epoch.append(val_loss)
        # 0: 1.145
        # 1: 1.65
        # 2: 1.05
        # 3: 0.987

    tqdm.write('Epoch [%d/%d], Average Train Loss: %.4f, Average Valiation Loss: %.4f'
                % (epoch + 1, EPOCH, train_loss, val_loss))

    if val_loss == np.min(val_loss_epoch):
        # current valida loss가 가장 작으면 best model로 저장하겠다
        print('Model saved')
        
        state = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        
        torch.save(state, os.path.join('best.pth.tar'))
        # torch.save(model.state_dict(), os.path.join('10e-best.pth.tar'))

  6%|▋         | 5/80 [00:14<03:34,  2.86s/it]

Epoch [1/100], Iter [5/80], Loss: 1.4117


 12%|█▎        | 10/80 [00:28<03:13,  2.76s/it]

Epoch [1/100], Iter [10/80], Loss: 1.0852


 19%|█▉        | 15/80 [00:41<02:57,  2.73s/it]

...

Epoch [5/100], Iter [5/80], Loss: 0.6152


 12%|█▎        | 10/80 [00:28<03:17,  2.82s/it]

Epoch [5/100], Iter [10/80], Loss: 0.6169


 16%|█▋        | 13/80 [00:38<03:18,  2.97s/it]


KeyboardInterrupt: 

In [19]:
# 데이터 샘플 추출
sample = validSet[0]
sample_img, sample_label = sample['img'], sample['label']
sample_img.size()

torch.Size([3, 320, 320])

In [10]:
class DiceScore(nn.Module):
    def __init__(self):
        super(DiceScore, self).__init__()

    def forward(self, img, label, smooth=1):
        
        img = F.sigmoid(img) # sigmoid를 통과한 출력이면 주석처리
        
        img = img.view(-1)
        label = label.view(-1)
        
        intersection = (img * label).sum()                            
        dice = (2.*intersection + smooth) / (img.sum() + label.sum() + smooth)  
        
        return dice 

In [11]:
#테스트 시작

# 1. 데이터 준비
image_dir   = './archive/2d_images/'
mask_dir    = './archive/2d_masks/'
test_txt   = './test.txt'

testSet = SegCTDataset(image = image_dir, mask=mask_dir, txt=train_txt, transforms=test_transform)
test_loader = DataLoader(testSet, batch_size=2)

# 2. 모델 준비
model = ResUNet()       # 쌩 모델
#torch.load()            # Best 모델 불러오기 
model.load_state_dict(torch.load('best.pth.tar')['state_dict'] )

# 3. 테스트시작
test_loss = []
test_score = []

for i, batch in enumerate(test_loader):
    img, label = batch['img'], batch['label']
    
    predict = model(img)
    #pred 예측값
    loss = LOSS_FUNC(predict, label)     # Dice score로 loss를 계산

    #Dice Score값 구하기
    smooth=1
    p=2

    prob_map = torch.nn.functional.sigmoid(predict.squeeze(0))
    predict = prob_map.argmax(0).detach().cpu()

    predict = predict.contiguous().view(predict.shape[0], -1)
    label = label.contiguous().view(label.shape[0], -1)

    num = torch.sum(torch.mul(predict, label), dim=1) + smooth
    den = torch.sum(predict.pow(p) + label.pow(p), dim=1) + smooth
    score = num/den
    score.mean()  

    test_loss.append(loss)
    test_score.append(score)

# 평균 값 계산
np.mean(test_loss)
np.mean(test_score)

: 

In [63]:
#테스트 시작

# 1. 데이터 준비
image_dir   = './archive/2d_images/'
mask_dir    = './archive/2d_masks/'
test_txt   = './test.txt'

testSet = SegCTDataset(image = image_dir, mask=mask_dir, txt=train_txt, transforms=test_transform)
test_loader = DataLoader(testSet, batch_size=4)

# 2. 모델 준비
model = ResUNet()       # 쌩 모델
#torch.load()            # Best 모델 불러오기 
model.load_state_dict(torch.load('best.pth.tar')['state_dict'] )

# 3. 테스트시작
test_loss = []
test_score = []

dice_score_calc = DiceScore()

for i, batch in enumerate(test_loader):
    img, label = batch['img'], batch['label']
    
    print(img.size())
    print(label.size())

    
    predict = model(img)
    loss = LOSS_FUNC(predict, label)     # Dice score로 loss를 계산
    test_loss.append(loss.item())

    # Calculate dice score
    dice_score = dice_score_calc(predict, label)  # Correct way to call it
    test_score.append(dice_score.item())

   
    test_loss.append(loss)
    # test_score.append(score)
    test_score.append(dice_score)

    # 평균 값 계산
    average_test_loss = np.mean(test_loss)
    average_test_score = np.mean(test_score)

TypeError: DiceScore.forward() missing 2 required positional arguments: 'img' and 'label'